<a href="https://colab.research.google.com/github/CindyXin97/MIT_COVID-19_Datathon/blob/master/County_level_Texas_California.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# County-level summaries (TX, CA)

This notebook merges different datasets into one summary tables

Main data sources:
+ UCLA jail releases and jail+prison covid informaiton
+ Vera Institute jail data by county
+ NYT covid data by county


Contents:
1. Initial Explorations
  + data cleaning and merging
2. Texas - Summary
  + county-level data (jail and overall county infection rates)
  + only has jail covid information for 10 counties
  + to do: add prison data
3. California - Summary
  + county-level data (jail and overall county infection rates)
  + only has jail covid information for LA county jails
  + to do: add prison data

# Initial Explorations

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from google.colab import files

## The Marshall Project Data

From https://github.com/themarshallproject/COVID_prison_data
+ State-by-state prison test, case, and deaths data (inmates and staff)

In [0]:
covid_prison_cases = pd.read_csv('https://raw.githubusercontent.com/themarshallproject/COVID_prison_data/master/data/covid_prison_cases.csv')
prison_populations = pd.read_csv('https://raw.githubusercontent.com/themarshallproject/COVID_prison_data/master/data/prison_populations.csv')
staff_populations = pd.read_csv('https://raw.githubusercontent.com/themarshallproject/COVID_prison_data/master/data/staff_populations.csv')

# drop notes columns & state name column
covid_prison_cases.drop(['notes'], axis = 1, inplace = True)
prison_populations.drop(['name', 'as_of_date'], axis = 1, inplace = True)
staff_populations.drop(['name', 'as_of_date', 'notes'], axis = 1, inplace = True)

In [74]:
covid_prison_cases.head()

,name,abbreviation,staff_tests,prisoner_tests,total_staff_cases,total_prisoner_cases,total_staff_deaths,total_prisoner_deaths,as_of_date
0,Alabama,AL,NaN,102.0,17.0,8.0,0.0,1.0,2020-05-06
1,Alaska,AK,NaN,235.0,8.0,1.0,0.0,0.0,2020-05-06
2,Arizona,AZ,NaN,277.0,52.0,69.0,0.0,5.0,2020-05-06
3,Arkansas,AR,NaN,NaN,127.0,944.0,0.0,5.0,2020-05-07
4,California,CA,NaN,1551.0,169.0,398.0,0.0,1.0,2020-05-06


In [75]:
# view most recent dates from each state + federal
#   ( max number of cases -> most recent )
idx0 = covid_prison_cases.groupby('name')['as_of_date'].transform(max) == covid_prison_cases['as_of_date']

covid_prison_cases_most_recent = covid_prison_cases[idx0]
covid_prison_cases_most_recent

,name,abbreviation,staff_tests,prisoner_tests,total_staff_cases,total_prisoner_cases,total_staff_deaths,total_prisoner_deaths,as_of_date
0,Alabama,AL,NaN,102.0,17.0,8.0,0.0,1.0,2020-05-06
1,Alaska,AK,NaN,235.0,8.0,1.0,0.0,0.0,2020-05-06
2,Arizona,AZ,NaN,277.0,52.0,69.0,0.0,5.0,2020-05-06
3,Arkansas,AR,NaN,NaN,127.0,944.0,0.0,5.0,2020-05-07
4,California,CA,NaN,1551.0,169.0,398.0,0.0,1.0,2020-05-06
5,Colorado,CO,NaN,1067.0,25.0,310.0,0.0,1.0,2020-05-07
6,Connecticut,CT,NaN,NaN,358.0,478.0,0.0,6.0,2020-05-06
7,Delaware,DE,301.0,267.0,64.0,125.0,0.0,3.0,2020-05-07
8,Florida,FL,NaN,805.0,180.0,393.0,0.0,7.0,2020-05-06
9,Georgia,GA,NaN,NaN,100.0,316.0,1.0,10.0,2020-05-06


In [0]:
# merge with one-shot data of inmate and staff populations
merged_tables_most_recent = covid_prison_cases_most_recent.join( 
    prison_populations.set_index('abbreviation'), 
    on='abbreviation', rsuffix='_prisonpop' 
    ).join( 
        staff_populations.set_index('abbreviation'),  
        on='abbreviation', how='outer', rsuffix='_staffpop')

# compute rates of testing, rate of infection, rate of deaths
merged_tables_most_recent['rate_prisoner_tests'] = merged_tables_most_recent['prisoner_tests'] / merged_tables_most_recent['april_pop']
merged_tables_most_recent['rate_prisoner_cases'] = merged_tables_most_recent['total_prisoner_cases'] / merged_tables_most_recent['april_pop']
merged_tables_most_recent['rate_prisoner_deaths'] = merged_tables_most_recent['total_prisoner_deaths'] / merged_tables_most_recent['total_prisoner_cases']

In [8]:
merged_tables_most_recent.sort_values( by = ['prisoner_tests'], ascending = False).head()

,name,abbreviation,staff_tests,prisoner_tests,total_staff_cases,total_prisoner_cases,total_staff_deaths,total_prisoner_deaths,as_of_date,april_pop,april_pop_staffpop,rate_prisoner_tests,rate_prisoner_cases,rate_prisoner_deaths
21,Michigan,MI,NaN,8113.0,293.0,2073.0,2.0,47.0,2020-05-06,37687,11963.0,0.215273,0.055006,0.022672
34,Ohio,OH,NaN,6922.0,483.0,4312.0,2.0,42.0,2020-05-06,48765,12192.0,0.141946,0.088424,0.009740
41,Tennessee,TN,NaN,5962.0,55.0,1959.0,0.0,1.0,2020-05-05,21150,5179.0,0.281891,0.092624,0.000510
42,Texas,TX,2379.0,2134.0,504.0,1336.0,5.0,23.0,2020-05-06,140124,36073.0,0.015229,0.009534,0.017216
20,Massachusetts,MA,NaN,1906.0,151.0,351.0,0.0,7.0,2020-05-06,8173,4679.0,0.233207,0.042946,0.019943


In [64]:
merged_tables_most_recent[(merged_tables_most_recent['name'] == 'Texas') | (merged_tables_most_recent['name'] == 'Michigan') | (merged_tables_most_recent['name'] == 'New York')]

,name,abbreviation,staff_tests,prisoner_tests,total_staff_cases,total_prisoner_cases,total_staff_deaths,total_prisoner_deaths,as_of_date,april_pop,april_pop_staffpop,rate_prisoner_tests,rate_prisoner_cases,rate_prisoner_deaths
21,Michigan,MI,NaN,8113.0,293.0,2073.0,2.0,47.0,2020-05-06,37687,11963.0,0.215273,0.055006,0.022672
31,New York,NY,NaN,619.0,1143.0,415.0,4.0,15.0,2020-05-06,42784,19123.0,0.014468,0.009700,0.036145
42,Texas,TX,2379.0,2134.0,504.0,1336.0,5.0,23.0,2020-05-06,140124,36073.0,0.015229,0.009534,0.017216


## Covid Prison Data

From https://covidprisondata.com/data.html
+ the facility data csv file

In [10]:
# for upload if run as google colab
uploaded = files.upload()

Saving covidprisondata_facility_historical.csv to covidprisondata_facility_historical.csv


In [0]:
facility_data = pd.read_csv('covidprisondata_facility_historical.csv')

In [12]:
facility_data.columns

Index(['facilities', 'state', 'scrape_date', 'staff_positive', 'staff_deaths',
       'staff_recovered', 'units_in_quarantine', 'inmates_quarantine',
       'housing_type', 'inmates_isolation', 'inmates_positive',
       'inmates_deaths_probable', 'inmates_deaths_confirmed',
       'inmates_pending', 'inmates_recovered', 'inmates_deaths',
       'inmates_negative', 'inmates_medical_restriction',
       'inmates_medical_isolation', 'inmates_tested', 'inmates_retested',
       'daily_total_population', 'inmates_security_quarantine',
       'inmates_medical_quarantine', 'inmates_positive_in_custody',
       'inmates_released_positive', 'inmates_positive_symptomatic',
       'inmates_positive_asymptomatic', 'inmates_positive_current',
       'inmates_step_down', 'inmates_covid_deaths_underlying_conditions',
       'inmates_covid_deaths', 'inmates_medical', 'staff_negative',
       'inmates_positive_on_site', 'inmates_hospital', 'location', 'county',
       'jail_inmates_negative', 'jail_in

In [14]:
facility_data[facility_data['state'] == "Texas"].sort_values(by = ['inmates_positive'], ascending = False).head(10)

,facilities,state,scrape_date,staff_positive,staff_deaths,staff_recovered,units_in_quarantine,inmates_quarantine,housing_type,inmates_isolation,inmates_positive,inmates_deaths_probable,inmates_deaths_confirmed,inmates_pending,inmates_recovered,inmates_deaths,inmates_negative,inmates_medical_restriction,inmates_medical_isolation,inmates_tested,inmates_retested,daily_total_population,inmates_security_quarantine,inmates_medical_quarantine,inmates_positive_in_custody,inmates_released_positive,inmates_positive_symptomatic,inmates_positive_asymptomatic,inmates_positive_current,inmates_step_down,inmates_covid_deaths_underlying_conditions,inmates_covid_deaths,inmates_medical,staff_negative,inmates_positive_on_site,inmates_hospital,location,county,jail_inmates_negative,jail_inmates_positive,jail_inmates_pending,jail_inmates_total,jail_inmates_quarantine,inmates_total,parolees_negative,parolees_positive,parolees_pending,parolees_total,parolees_quarantine,juveniles_negative,juveniles_positive,juveniles_pending,juveniles_total,juveniles_quarantine,recovered,contract_staff_positive,staff_positive_asymptomatic,staff_positive_symptomatic,facility_type,staff_tested,tier_status,inmates_recovered_in_previous_calendar_day,inmates_released
152,Murray,Texas,2020-05-10,20.0,NaN,NaN,NaN,NaN,NaN,NaN,68.0,NaN,NaN,7.0,NaN,NaN,21.0,594.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,Ellis,Texas,2020-05-10,21.0,NaN,NaN,NaN,NaN,NaN,NaN,64.0,NaN,NaN,8.0,NaN,NaN,16.0,1304.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,Beto,Texas,2020-05-10,20.0,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,19.0,NaN,NaN,51.0,2349.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Estelle,Texas,2020-05-10,17.0,NaN,NaN,NaN,NaN,NaN,NaN,52.0,NaN,NaN,6.0,NaN,NaN,62.0,893.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,Scott,Texas,2020-05-10,13.0,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,15.0,NaN,NaN,8.0,537.0,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,Michael,Texas,2020-05-10,25.0,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,8.0,NaN,NaN,19.0,964.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,Clements,Texas,2020-05-10,60.0,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,2.0,NaN,NaN,8.0,961.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,Wynne,Texas,2020-05-10,63.0,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,3.0,NaN,NaN,18.0,2156.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,Middleton,Texas,2020-05-10,9.0,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,14.0,NaN,NaN,9.0,806.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,Hospital Galveston,Texas,2020-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,NaN,NaN,5.0,NaN,NaN,11.0,0.0,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
facility_data[facility_data['state'] == "Florida"]

,facilities,state,scrape_date,staff_positive,staff_deaths,staff_recovered,units_in_quarantine,inmates_quarantine,housing_type,inmates_isolation,inmates_positive,inmates_deaths_probable,inmates_deaths_confirmed,inmates_pending,inmates_recovered,inmates_deaths,inmates_negative,inmates_medical_restriction,inmates_medical_isolation,inmates_tested,inmates_retested,daily_total_population,inmates_security_quarantine,inmates_medical_quarantine,inmates_positive_in_custody,inmates_released_positive,inmates_positive_symptomatic,inmates_positive_asymptomatic,inmates_positive_current,inmates_step_down,inmates_covid_deaths_underlying_conditions,inmates_covid_deaths,inmates_medical,staff_negative,inmates_positive_on_site,inmates_hospital,location,county,jail_inmates_negative,jail_inmates_positive,jail_inmates_pending,jail_inmates_total,jail_inmates_quarantine,inmates_total,parolees_negative,parolees_positive,parolees_pending,parolees_total,parolees_quarantine,juveniles_negative,juveniles_positive,juveniles_pending,juveniles_total,juveniles_quarantine,recovered,contract_staff_positive,staff_positive_asymptomatic,staff_positive_symptomatic,facility_type,staff_tested,tier_status,inmates_recovered_in_previous_calendar_day,inmates_released
282,Apalachee CI,Florida,2020-05-10,11.0,NaN,NaN,NaN,NaN,NaN,1.0,50.0,NaN,NaN,1002.0,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,20.0,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,Avon Park CI,Florida,2020-05-10,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,Baker CI,Florida,2020-05-10,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,16.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
285,Bay CF Operated by The GEO Group Inc.,Florida,2020-05-10,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,Blackwater CF Operated by The GEO Group Inc.,Florida,2020-05-10,11.0,NaN,NaN,NaN,NaN,NaN,0.0,57.0,NaN,NaN,0.0,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,3.0,242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,Zephyrhills CI,Florida,2020-05-10,4.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,1.0,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,2.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,Community Corrections Region 1,Florida,2020-05-10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,Community Corrections Region 2,Florida,2020-05-10,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,Community Corrections Region 3,Florida,2020-05-10,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

## Prison Releases

From the UCLA spreadsheet
+ covid_prison_releases.csv

In [262]:
# for upload if run as google colab
uploaded = files.upload()

Saving covid_prison_releases.csv to covid_prison_releases.csv


In [0]:
pr = pd.read_csv('covid_prison_releases.csv')

In [264]:
pr.head()

,State,Facility,Authorizing Agent,Known Capacity \n(if available),Population Prior to Releases,Overall Pop. Reduction / \nTotal Number of Releases,Does the source report this reduction as a result of releases only or a mix of releases/policy changes?,Does the source give a more detailed breakdown on the releases?,Breakdown of Releases:\nParole Tech Violation,Breakdown of Releases:\nLower Level/ Non-Violent Crimes,Breakdown of Releases: \nShort Time Left on Sentence,Breakdown of Releases:\nVulnerable Populations,Breakdown of Releases: \nOther (please explain),Date,Legal Filing?,Source(s),"Additional Notes (explaining ""other"" column, defining vulnerable populations in this instance, etc.)",Unnamed: 17
0,NaN,NaN,NaN,NaN,587796,17452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,Statewide,Governor,NaN,NaN,Unknown,Releases only,Yes - See columns I-M,x,NaN,NaN,NaN,x,4/2/20,No,https://www.wsfa.com/2020/04/02/ivey-order-all...,County jail releases only (no prisons); focus ...,NaN
2,California,Statewide,CDCR,NaN,114000,3418,Releases only,Yes - See columns I-M,NaN,x,x,NaN,NaN,4/13/20,Yes,https://www.courtlistener.com/recap/gov.uscour...,This is a declaration opposing further relief ...,NaN
3,Colorado,Statewide,Governor,NaN,17000,52,Releases only,No,NaN,NaN,NaN,NaN,NaN,4/15/20,No,https://www.denverpost.com/2020/04/13/colorado...,52 early release pursuant to a governor order,NaN
4,Federal BOP,Nationwide,AG,NaN,174000,1440,Releases only,Yes - See columns I-M,NaN,x,NaN,x,NaN,4/22/20,No,https://www.politico.com/amp/news/2020/04/22/c...,Number as of 4.22.20,(As of 4/6/20: 886 had been moved to home re...


## Jail data by county

Data sources:
+ UCLA Spreadsheet
  + covid jail releases
  + covid facility data
+ NYTimes County-level data
+ Vera Institute county jail data

### UCLA data

From the UCLA spreadsheet
  + jail_prison_confirmed_cases.csv
    + for Texas: See TCJS data in a latter section
  + covid_jail_releases.csv

In [15]:
# for upload if run as google colab
uploaded = files.upload()

Saving covid_jail_releases.csv to covid_jail_releases.csv
Saving jail_prison_confirmed_cases.csv to jail_prison_confirmed_cases.csv


#### Jail/Prison Confirmed Cases

In [77]:
jail_prison_confirmed_cases = pd.read_csv('jail_prison_confirmed_cases.csv')

# filter jail information only
#jail_prison_confirmed_cases[(jail_prison_confirmed_cases["Facility"] == "Jail")]
jail_prison_confirmed_cases#[(jail_prison_confirmed_cases["Facility"] == "Jail")]

,Facility,State,County / Name of Facility,Confirmed Cases \n(Staff),Confirmed Cases (Residents),Confirmed Deaths\n(Staff),Confirmed Deaths\n(Residents),Staff Recovered,Residents Recovered,Date \n(last updated),Source(s),Additional Notes,Coder,Unnamed: 13
0,NaN,NaN,NaN,8460,25694.0,37.0,363.0,2037.0,7531.0,NaN,NaN,NaN,NaN,NaN
1,Prison,Alabama,State-Wide (for non-specific facilities),NaN,NaN,NaN,NaN,NaN,NaN,5/11/20,http://www.doc.alabama.gov/covid19news,(4/25/20) update on website from 4/24 says 2 s...,KP,NaN
2,Prison,Alabama,Alex City,1,0.0,0.0,0.0,0.0,0.0,5/11/20,http://www.doc.alabama.gov/covid19news,(4/25/20) update on website from 4/24 says 2 s...,KP,NaN
3,Prison,Alabama,ATEF,0,0.0,0.0,0.0,0.0,0.0,5/11/20,http://www.doc.alabama.gov/covid19news,(4/25/20) update on website from 4/24 says 2 s...,KP,NaN
4,Prison,Alabama,Bibb,0,1.0,0.0,0.0,0.0,0.0,5/11/20,http://www.doc.alabama.gov/covid19news,(4/25/20) update on website from 4/24 says 2 s...,KP,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,Jail,California,Los Angeles Jails,107,357.0,0.0,0.0,NaN,117.0,5/11/20,https://lasd.org/covid19updates/,(4/23/20) total of 44 inmates positive to date...,GD,NaN
1105,Jail,District of Columbia,County-Wide,74,177.0,1.0,1.0,37.0,127.0,5/11/20,https://coronavirus.dc.gov/page/public-safety-...,(4/13/20) One DC death reported ( https://www....,GD,NaN
1106,Jail,Pennsylvania,Philadelphia Dept. of Prisons,NaN,196.0,NaN,NaN,NaN,184.0,5/11/20,https://www.phila.gov/programs/coronavirus-dis...,(5/11/20) the Philadelphia prison page now red...,GD,NaN
1107,Jail,Texas,Texas Commission on Jail Standards,338,1255.0,NaN,2.0,NaN,NaN,5/11/20,https://www.tcjs.state.tx.us,(5/5/20) Bexar County jail resident dies of Co...,GD,NaN


In [17]:
jail_prison_confirmed_cases[(jail_prison_confirmed_cases["Facility"] == "Jail")]

,Facility,State,County / Name of Facility,Confirmed Cases \n(Staff),Confirmed Cases (Residents),Confirmed Deaths\n(Staff),Confirmed Deaths\n(Residents),Staff Recovered,Residents Recovered,Date \n(last updated),Source(s),Additional Notes,Coder,Unnamed: 13
876,Jail,Utah,UDC Inmates Housed at a County Jail,NaN,0.0,NaN,0.0,NaN,0.0,5/11/20,https://corrections.utah.gov/index.php/home/al...,(4/22/20) Utah released a facility-specific tr...,GD,NaN
1102,Jail,New York,New York City Jails,1279,371.0,9.0,3.0,NaN,NaN,5/11/20,https://www1.nyc.gov/site/boc/covid-19.page,(5/5/20) articel reports nine Rikers Island st...,GD,NaN
1103,Jail,Illinois,Cook County,396,534.0,3.0,7.0,281.0,344.0,5/11/20,https://www.cookcountysheriff.org/covid-19-cas...,"(4/26/20) 229 residents currently positive, 23...",GD,NaN
1104,Jail,California,Los Angeles Jails,107,357.0,0.0,0.0,NaN,117.0,5/11/20,https://lasd.org/covid19updates/,(4/23/20) total of 44 inmates positive to date...,GD,NaN
1105,Jail,District of Columbia,County-Wide,74,177.0,1.0,1.0,37.0,127.0,5/11/20,https://coronavirus.dc.gov/page/public-safety-...,(4/13/20) One DC death reported ( https://www....,GD,NaN
1106,Jail,Pennsylvania,Philadelphia Dept. of Prisons,NaN,196.0,NaN,NaN,NaN,184.0,5/11/20,https://www.phila.gov/programs/coronavirus-dis...,(5/11/20) the Philadelphia prison page now red...,GD,NaN
1107,Jail,Texas,Texas Commission on Jail Standards,338,1255.0,NaN,2.0,NaN,NaN,5/11/20,https://www.tcjs.state.tx.us,(5/5/20) Bexar County jail resident dies of Co...,GD,NaN
1108,Jail,Louisiana,Orleans Parish Jails,82,131.0,3.0,0.0,25.0,NaN,5/11/20,http://opcso.org/,(4/28/20) Staff number includes 68 OPSO employ...,GD,NaN


In [242]:
california_jail_covid = jail_prison_confirmed_cases[(jail_prison_confirmed_cases["Facility"] == "Jail") & (jail_prison_confirmed_cases["State"] == "California")]

california_jail_covid.rename( columns={'Facility':'facility',
                                       'State':'state',
                                       'County / Name of Facility':'county',
                                       'Confirmed Cases \n(Staff)':'staff_confirmed',
                                       'Confirmed Cases (Residents)':'inmates_confirmed',
                                       'Confirmed Deaths\n(Staff)':'staff_deaths',
                                       'Confirmed Deaths\n(Residents)':'inmates_deaths',
                                       'Staff Recovered':'staff_recovered',
                                       'Residents Recovered':'inmates_recovered',
                                       'Date \n(last updated)':'date'}, 
                             inplace= True)

# select most relevant columns
california_jail_covid = california_jail_covid[['facility','state', 'county', 'date', 'staff_confirmed', 'inmates_confirmed', 'staff_deaths', 'inmates_deaths', 'staff_recovered', 'inmates_recovered']]

#rename 'Los Angeles County Jailes' in the county column to just 'Los Angeles'
california_jail_covid.iloc[0, 1] = 'Los Angeles'

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [218]:
california_jail_covid

,state,county,date,staff_confirmed,inmates_confirmed,staff_deaths,inmates_deaths,staff_recovered,inmates_recovered
1104,California,Los Angeles,5/11/20,107,357.0,0.0,0.0,NaN,117.0


In [257]:
# another state

XX_jail_prison_covid = jail_prison_confirmed_cases[ (jail_prison_confirmed_cases["State"] == "Ohio")]

XX_jail_prison_covid.rename( columns={'Facility':'facility',
                                       'State':'state',
                                       'County / Name of Facility':'name',
                                       'Confirmed Cases \n(Staff)':'staff_confirmed',
                                       'Confirmed Cases (Residents)':'inmates_confirmed',
                                       'Confirmed Deaths\n(Staff)':'staff_deaths',
                                       'Confirmed Deaths\n(Residents)':'inmates_deaths',
                                       'Staff Recovered':'staff_recovered',
                                       'Residents Recovered':'inmates_recovered',
                                       'Date \n(last updated)':'date'}, 
                             inplace= True)

# select most relevant columns
XX_jail_prison_covid = XX_jail_prison_covid[['facility','state', 'name', 'date', 'staff_confirmed', 'inmates_confirmed', 'staff_deaths', 'inmates_deaths', 'staff_recovered', 'inmates_recovered']]


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [258]:
XX_jail_prison_covid

,facility,state,name,date,staff_confirmed,inmates_confirmed,staff_deaths,inmates_deaths,staff_recovered,inmates_recovered
613,Prison,Ohio,Allen Oakwood CI,5/11/20,0,0.0,0.0,0.0,0.0,0.0
614,Prison,Ohio,Belmont CI,5/11/20,42,119.0,0.0,0.0,4.0,27.0
615,Prison,Ohio,Chillicothe CI,5/11/20,1,6.0,0.0,0.0,0.0,0.0
616,Prison,Ohio,Correctional Reception Center,5/11/20,18,76.0,0.0,1.0,7.0,65.0
617,Prison,Ohio,Dayton CI,5/11/20,1,1.0,0.0,0.0,1.0,1.0
618,Prison,Ohio,Franklin Medical Center,5/11/20,87,223.0,0.0,4.0,57.0,94.0
619,Prison,Ohio,Grafton CI,5/11/20,0,1.0,0.0,0.0,0.0,1.0
620,Prison,Ohio,Lake Erie CI,5/11/20,6,12.0,0.0,0.0,0.0,0.0
621,Prison,Ohio,Lebanon CI,5/11/20,3,0.0,0.0,0.0,2.0,0.0
622,Prison,Ohio,London CI,5/11/20,6,18.0,0.0,1.0,1.0,0.0


#### Jail Releases

In [0]:
# from Jessica's jupyter notebook
jr = pd.read_csv('covid_jail_releases.csv')
jr = jr.iloc[1:,:7] # jail releases

# rename columns for ease of use
jr.rename(columns={'Overall Pop. Reduction / \nTotal Number of Releases':'releases',
                  'Population Prior to Releases':'prior_pop'},
          inplace=True)

# some rows have weird entries (range of values, circa values), remove
jr = jr[jr['releases'].astype(str).apply(lambda x: x.isnumeric())]
jr = jr[jr['prior_pop'].astype(str).apply(lambda x: x.isnumeric())]

# make columns numeric
jr['releases'] = pd.to_numeric(jr['releases'])
jr['prior_pop'] = pd.to_numeric(jr['prior_pop'])

# what fraction of the prior population has been released
jr['frac_reduction'] = jr['releases']/jr['prior_pop']

In [19]:
jr.sort_values('frac_reduction',ascending = False)

,State,County,Facility,Authorizing Agent,Known Capacity \n(if available),prior_pop,releases,frac_reduction
28,District of Col,DC jail,DC jail,NaN,NaN,1850,1442,0.779459
76,Oregon,Marion,County-wide,NaN,NaN,415,300,0.722892
41,Iowa,Polk,County-wide,NaN,NaN,1000,700,0.700000
77,Oregon,Multnomah,County-wide,"Sheriff, courts",1192.0,1097,739,0.673655
73,Oregon,Douglas,County-wide,NaN,NaN,194,128,0.659794
...,...,...,...,...,...,...,...,...
57,Nevada,Clark County,County-wide,NaN,NaN,2900,115,0.039655
60,New York,NYC,City-wide,Governor,NaN,5000,106,0.021200
6,Arizona,Pima County,County-wide,Court,2300.0,1760,27,0.015341
62,New York,NYC,City-wide,Courts,NaN,4300,51,0.011860


In [0]:
# filter only texas jails; take most relevant columns
texas_jail_releases = jr[jr['State'] == 'Texas']

# get rid of white spaces in the County column
texas_jail_releases.insert(1, 'county', texas_jail_releases['County'].str.strip() )
texas_jail_releases = texas_jail_releases.drop('County', axis = 1)

In [30]:
texas_jail_releases

,State,county,Facility,Authorizing Agent,Known Capacity \n(if available),prior_pop,releases,frac_reduction
87,Texas,Dallas,County-wide,Courts,NaN,5879,1000,0.170097
88,Texas,Harris,County-wide,NaN,NaN,9059,1789,0.197483
89,Texas,Travis,County-wide,courts,NaN,2164,97,0.044824


In [0]:
# filter only california jails; take most relevant columns
california_jail_releases = jr[jr['State'] == 'California']

# get rid of white spaces in the County column
california_jail_releases.insert(1, 'county', california_jail_releases['County'].str.strip() )
california_jail_releases = california_jail_releases.drop('County', axis = 1)

In [118]:
california_jail_releases

,State,county,Facility,Authorizing Agent,Known Capacity \n(if available),prior_pop,releases,frac_reduction
12,California,Alameda,Santa Rita,Court,NaN,2500,247,0.098800
13,California,Los Angeles,County-wide,NaN,NaN,17076,5193,0.304111
16,California,Alameda,NaN,NaN,NaN,2500,500,0.200000
17,California,San Francisco,County-wide,Joint Initiative,NaN,1100,260,0.236364
19,California,San Diego,Ocean View,NaN,NaN,55,34,0.618182
20,California,Marin,County jail,Court/others,NaN,270,58,0.214815
21,California,Monterey,County-wide,NaN,NaN,885,244,0.275706
23,California,San Mateo,County-wide,NaN,1500.0,969,382,0.394221
25,California,Orange,County-wide,Court,NaN,5289,838,0.158442


### NYTimes County Data

+ https://github.com/nytimes/covid-19-data/blob/master/us-counties.csv

In [0]:
us_counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

#### Texas

In [0]:

# filter texas only
texas_pop_covid = us_counties[(us_counties["state"] == "Texas")]

# get most recent data
idx = texas_pop_covid.groupby(['county'])['date'].transform(max) == texas_pop_covid['date']
texas_pop_covid_mostrecent = texas_pop_covid[idx]

# drop some other  redundant columns
texas_pop_covid_mostrecent = texas_pop_covid_mostrecent.drop(['state', 'fips'], axis=1)

In [35]:
texas_pop_covid_mostrecent.head()

,date,county,cases,deaths
135000,2020-05-11,Anderson,41,0
135001,2020-05-11,Andrews,21,0
135002,2020-05-11,Angelina,100,0
135003,2020-05-11,Aransas,2,0
135004,2020-05-11,Armstrong,2,0


#### California

In [0]:
# filter california only
california_pop_covid = us_counties[(us_counties["state"] == "California")]

# get most recent data
idx_california = california_pop_covid.groupby(['county'])['date'].transform(max) == california_pop_covid['date']
california_pop_covid_mostrecent = california_pop_covid[idx_california]

# drop some other  redundant columns
california_pop_covid_mostrecent = california_pop_covid_mostrecent.drop(['state', 'fips'], axis=1)

In [108]:
california_pop_covid_mostrecent

,date,county,cases,deaths
127056,2020-05-09,Unknown,2,0
132829,2020-05-11,Alameda,2122,75
132830,2020-05-11,Alpine,2,0
132831,2020-05-11,Amador,7,0
132832,2020-05-11,Butte,20,0
132833,2020-05-11,Calaveras,13,0
132834,2020-05-11,Colusa,3,0
132835,2020-05-11,Contra Costa,1048,32
132836,2020-05-11,Del Norte,3,0
132837,2020-05-11,El Dorado,56,0


#### Ohio

In [0]:
# filter ohio only
ohio_pop_covid = us_counties[(us_counties["state"] == "Ohio")]

# get most recent data
idx_ohio = ohio_pop_covid.groupby(['county'])['date'].transform(max) == ohio_pop_covid['date']
ohio_pop_covid_mostrecent = ohio_pop_covid[idx_ohio]

# drop some other  redundant columns
ohio_pop_covid_mostrecent = ohio_pop_covid_mostrecent.drop(['state', 'fips'], axis=1)

In [268]:
ohio_pop_covid_mostrecent

,date,county,cases,deaths
13508,2020-03-26,Unknown,1,0
134545,2020-05-11,Adams,6,0
134546,2020-05-11,Allen,146,29
134547,2020-05-11,Ashland,15,0
134548,2020-05-11,Ashtabula,193,19
...,...,...,...,...
134628,2020-05-11,Washington,114,17
134629,2020-05-11,Wayne,188,42
134630,2020-05-11,Williams,44,1
134631,2020-05-11,Wood,225,31


#### Oregon

In [0]:
# filter oregon only
oregon_pop_covid = us_counties[(us_counties["state"] == "Oregon")]

# get most recent data
idx_oregon = oregon_pop_covid.groupby(['county'])['date'].transform(max) == oregon_pop_covid['date']
oregon_pop_covid_mostrecent = oregon_pop_covid[idx_oregon]

# drop some other  redundant columns
oregon_pop_covid_mostrecent = oregon_pop_covid_mostrecent.drop(['state', 'fips'], axis=1)

In [279]:
oregon_pop_covid_mostrecent

,date,county,cases,deaths
134704,2020-05-11,Baker,1,0
134705,2020-05-11,Benton,49,5
134706,2020-05-11,Clackamas,268,9
134707,2020-05-11,Clatsop,33,0
134708,2020-05-11,Columbia,15,0
134709,2020-05-11,Coos,30,0
134710,2020-05-11,Crook,1,0
134711,2020-05-11,Curry,4,0
134712,2020-05-11,Deschutes,86,0
134713,2020-05-11,Douglas,24,0


### Vera Institute 

Jail data by county
+ https://github.com/vera-institute/jail-population-data/blob/master/jail_population.csv



In [0]:
jail_population = pd.read_csv('https://raw.githubusercontent.com/vera-institute/jail-population-data/master/jail_population.csv')

#rename column"state_name" to "state" to match NYTimes data
jail_population.rename(columns={'state_name':'state'}, inplace = True)

#### Texas

In [0]:
# filter texas only
texas_jails = jail_population[jail_population["state"] == "Texas"]

# get most recent data
idx2 = texas_jails.groupby(['county_name'])['date'].transform(max) == texas_jails['date']
texas_jails_mostrecent = texas_jails[idx2]

# rename each entry in "Abc County" as "Abc" (remove the word County at the end, to match NYTimes data)
#     keep name in new column called county
texas_jails_mostrecent.insert(4, 'county', texas_jails_mostrecent['county_name'].str.split(' ').str[0])
#     drop the old county_name column
texas_jails_mostrecent = texas_jails_mostrecent.drop('county_name', axis = 1)

In [42]:
texas_jails_mostrecent.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k
65622,48001,2020-05-01,138.0,Anderson,Texas,rural,Anderson County Sheriff's Office,57735.0,239.0
65627,48003,2020-05-01,28.0,Andrews,Texas,rural,Andrews County Jail,18705.0,149.7
65652,48005,2020-05-01,160.0,Angelina,Texas,rural,Angelina County Sheriffs Department,86715.0,184.5
65657,48007,2020-05-01,140.0,Aransas,Texas,small/mid,Aransas County Jail,23510.0,595.5
65761,48009,2020-05-01,20.0,Archer,Texas,small/mid,Archer County Jail,8553.0,233.8


#### California

In [0]:
# filter california only
california_jails = jail_population[jail_population["state"] == "California"]

# get most recent data
idx2_california = california_jails.groupby(['county_name'])['date'].transform(max) == california_jails['date']
california_jails_mostrecent = california_jails[idx2_california]

# rename each entry in "Abc County" as "Abc" (remove the word County at the end, to match NYTimes data)
#     keep name in new column called county
california_jails_mostrecent.insert(4, 'county', california_jails_mostrecent['county_name'].str.rsplit(' ', n = 1).str[0])
#     drop the old county_name column
california_jails_mostrecent = california_jails_mostrecent.drop('county_name', axis = 1)

In [174]:
california_jails_mostrecent.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k
6358,6001,2020-04-30,1757.0,Alameda,California,urban,Alameda County Sheriffs Office,1671329.0,105.1
6403,6005,2020-05-01,52.0,Amador,California,rural,Amador County Sheriff's Office,39752.0,130.8
6514,6017,2020-05-11,307.0,El Dorado,California,suburban,El Dorado County Sheriff's Office,192843.0,159.2
6551,6019,2020-05-11,2250.0,Fresno,California,small/mid,Fresno County Sheriff Coroner's Office,999101.0,225.2
6668,6031,2020-05-11,436.0,Kings,California,small/mid,Kings County Sheriffs Office,152940.0,285.1


#### Ohio

In [0]:
# filter ohio only
ohio_jails = jail_population[jail_population["state"] == "Ohio"]

# get most recent data
idx2_ohio = ohio_jails.groupby(['county_name'])['date'].transform(max) == ohio_jails['date']
ohio_jails_mostrecent = ohio_jails[idx2_ohio]

# rename each entry in "Abc County" as "Abc" (remove the word County at the end, to match NYTimes data)
#     keep name in new column called county
ohio_jails_mostrecent.insert(4, 'county', ohio_jails_mostrecent['county_name'].str.rsplit(' ', n = 1).str[0])
#     drop the old county_name column
ohio_jails_mostrecent = ohio_jails_mostrecent.drop('county_name', axis = 1)

In [271]:
ohio_jails_mostrecent.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k
56931,39001,2020-05-12,20.0,Adams,Ohio,rural,Adams County Jail,27698.0,72.2
56976,39009,2020-05-12,126.0,Athens,Ohio,rural,Southeastern Ohio Regional Jail,65327.0,192.9
57002,39013,2020-05-01,106.0,Belmont,Ohio,small/mid,Belmont County Jail,67006.0,158.2
57103,39025,2020-05-01,255.0,Clermont,Ohio,suburban,Clermont County Sheriffs Office,206428.0,123.5
57217,39027,2020-05-01,50.0,Clinton,Ohio,rural,Clinton County Jail,41968.0,119.1


#### Oregon

In [0]:
# filter oregon only
oregon_jails = jail_population[jail_population["state"] == "Oregon"]

# get most recent data
idx2_oregon = oregon_jails.groupby(['county_name'])['date'].transform(max) == oregon_jails['date']
oregon_jails_mostrecent = oregon_jails[idx2_oregon]

# rename each entry in "Abc County" as "Abc" (remove the word County at the end, to match NYTimes data)
#     keep name in new column called county
oregon_jails_mostrecent.insert(4, 'county', oregon_jails_mostrecent['county_name'].str.rsplit(' ', n = 1).str[0])
#     drop the old county_name column
oregon_jails_mostrecent = oregon_jails_mostrecent.drop('county_name', axis = 1)

In [281]:
oregon_jails_mostrecent.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k
59885,41001,2020-05-01,14.0,Baker,Oregon,rural,Baker County Jail,16124.0,86.8
59933,41003,2020-02-17,53.0,Benton,Oregon,small/mid,Benton County Sheriffs Office,93053.0,57.0
60020,41005,2020-05-01,137.0,Clackamas,Oregon,suburban,Clackamas County Sheriffs Office,418187.0,32.8
60139,41007,2020-05-01,32.0,Clatsop,Oregon,rural,Clatsop County Corrections Facility,40224.0,79.6
60231,41009,2020-05-01,92.0,Columbia,Oregon,suburban,Columbia County Jail,52354.0,175.7


#### Other Vera Institute datasets

[This NYTimes article](https://www.nytimes.com/2020/03/31/upshot/coronavirus-jails-prisons.html) mentioned this as their datasource, but I can't find their weekly admit/release data here

https://github.com/vera-institute/incarceration_trends/blob/master/incarceration_trends.csv

(Haven't done much with these yet)

In [0]:
vera2 = pd.read_csv('https://raw.githubusercontent.com/vera-institute/incarceration_trends/master/incarceration_trends.csv')

In [96]:
vera2[(vera2['state']=='TX') & (vera2['year'] >= 2000 )].head()

,yfips,year,fips,state,county_name,total_pop,total_pop_15to64,female_pop_15to64,male_pop_15to64,aapi_pop_15to64,black_pop_15to64,latinx_pop_15to64,native_pop_15to64,white_pop_15to64,urbanicity,region,division,commuting_zone,metro_area,land_area,total_jail_pop,female_jail_pop,male_jail_pop,female_adult_jail_pop,female_juvenile_jail_pop,male_adult_jail_pop,male_juvenile_jail_pop,aapi_jail_pop,black_jail_pop,latinx_jail_pop,native_jail_pop,white_jail_pop,other_race_jail_pop,total_jail_pretrial,total_jail_from_prison,total_jail_from_other_jail,total_jail_from_fed,total_jail_from_bia,total_jail_from_bop,total_jail_from_ice,...,other_race_prison_adm,white_prison_adm,aapi_female_prison_adm,aapi_male_prison_adm,black_female_prison_adm,black_male_prison_adm,latinx_female_prison_adm,latinx_male_prison_adm,native_female_prison_adm,native_male_prison_adm,other_race_female_prison_adm,other_race_male_prison_adm,white_female_prison_adm,white_male_prison_adm,total_jail_pop_rate,female_jail_pop_rate,male_jail_pop_rate,aapi_jail_pop_rate,black_jail_pop_rate,latinx_jail_pop_rate,native_jail_pop_rate,white_jail_pop_rate,total_jail_adm_rate,total_jail_pretrial_rate,total_prison_pop_rate,female_prison_pop_rate,male_prison_pop_rate,aapi_prison_pop_rate,black_prison_pop_rate,latinx_prison_pop_rate,native_prison_pop_rate,white_prison_pop_rate,total_prison_adm_rate,female_prison_adm_rate,male_prison_adm_rate,aapi_prison_adm_rate,black_prison_adm_rate,latinx_prison_adm_rate,native_prison_adm_rate,white_prison_adm_rate
120990,200048001,2000,48001,TX,Anderson County,55062,39319,13213,26106,199.0,10167.0,5267.0,113.0,23573.0,rural,South,West South Central,475.0,37300.0,1062.6,112.0,15.0,100.0,15.0,0.0,100.0,0.0,0.0,60.0,22.0,0.0,33.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,284.85,113.52,383.05,0.0,590.14,417.70,0.0,139.99,6348.08,76.30,1299.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,389.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120991,200148001,2001,48001,TX,Anderson County,54263,38531,13285,25246,212.0,9708.0,5156.0,117.0,23338.0,rural,South,West South Central,475.0,37300.0,1062.6,105.0,18.0,100.0,18.0,0.0,99.0,1.0,0.0,44.0,10.0,0.0,64.0,0.0,57.0,9.0,10.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.51,135.49,396.10,0.0,453.23,193.95,0.0,274.23,1079.65,147.93,1394.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,419.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120992,200248001,2002,48001,TX,Anderson County,54740,38851,13493,25358,194.0,9706.0,5346.0,123.0,23482.0,rural,South,West South Central,475.0,37300.0,1062.6,116.0,10.0,101.0,10.0,0.0,101.0,0.0,0.0,51.0,9.0,0.0,51.0,0.0,95.0,17.0,9.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298.58,74.11,398.30,0.0,525.45,168.35,0.0,217.19,7495.30,244.52,1450.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,438.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120993,200348001,2003,48001,TX,Anderson County,56068,40125,13686,26439,201.0,10097.0,5693.0,130.0,24004.0,rural,South,West South Central,475.0,37300.0,1062.6,115.0,23.0,125.0,23.0,0.0,125.0,0.0,0.0,64.0,7.0,0.0,77.0,0.0,82.0,17.0,2.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286.60,168.05,472.79,0.0,633.85,122.96,0.0,320.78,10497.20,204.36,1469.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,445.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120994,200448001,2004,48001,TX,Anderson County,56245,40323,13690,26633,204.0,10087.0,5844.0,125.0,24063.0,rural,South,West South Central,475.0,37300.0,1062.6,106.0,19.0,109.0,19.0,0.0,108.0,1.0,0.0,61.0,5.0,0.0,62.0,0.0,94.0,0.0,4.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,262.88,138.79,409.27,0.0,604.74,85.56,0.0,257.66,7221.68,233.12,1527.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,464.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN


https://github.com/vera-institute/incarceration_trends/blob/master/incarceration_trends_jail_jurisdiction.csv

In [0]:
vera3 = pd.read_csv('https://raw.githubusercontent.com/vera-institute/incarceration_trends/master/incarceration_trends_jail_jurisdiction.csv')

In [98]:
vera3.head()

,Unnamed: 0,yjid,jid,fips,total_jail_pop,female_jail_pop,male_jail_pop,female_juvenile_jail_pop,male_juvenile_jail_pop,aapi_jail_pop,black_jail_pop,latinx_jail_pop,native_jail_pop,white_jail_pop,other_race_jail_pop,total_jail_pretrial,total_jail_from_prison,total_jail_from_other_jail,total_jail_from_fed,total_jail_from_bia,total_jail_from_bop,total_jail_from_ice,total_jail_from_marshals,total_jail_from_other_fed,jail_rated_capacity,private_jail_flag,regional_jail_flag,total_jail_adm,total_jail_dis,year,jurisdiction_name
0,0,197001001001,1001001,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970,AUTAUGA METRO JAIL
1,1,197101001001,1001001,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1971,AUTAUGA METRO JAIL
2,2,197201001001,1001001,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1972,AUTAUGA METRO JAIL
3,3,197301001001,1001001,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1973,AUTAUGA METRO JAIL
4,4,197401001001,1001001,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1974,AUTAUGA METRO JAIL


### Texas jail covid data (from Texas Commision for Jail Standards)
Texas
county-level jail covid data: https://www.tcjs.state.tx.us/wp-content/uploads/2020/05/TCJS_COVID_Report.pdf
  + texas_jail_covid.csv

In [51]:
# for upload if run as google colab
uploaded = files.upload()

Saving texas_jail_covid.csv to texas_jail_covid.csv


In [0]:
texas_jail_covid = pd.read_csv('texas_jail_covid.csv')

In [0]:
texas_jail_covid.insert(0, 'facility', 'Jail')

In [246]:
texas_jail_covid.head()

,facility,county,date,inmates_active_positive_test_confirmation,inmates_pending_test_results,inmates_quarantined,inmates_treated_offside,inmates_deaths_confirmed,inmates_deaths_suspected,staff_active_positive_test,staff_quarantined_pending_test_results
0,Jail,Bexar,2020-05-12,208,124,1185,0,1,0,41,581
1,Jail,Dallas,2020-05-12,233,9,824,0,0,0,47,13
2,Jail,Fannin,2020-05-12,2,0,25,0,0,0,0,1
3,Jail,Gregg,2020-05-12,4,0,19,0,0,0,2,0
4,Jail,Harris,2020-05-12,669,5,2511,14,0,1,193,56


### Merges 


#### Vera Institute + NYTimes Data

In [0]:
# texas
# merge
texas_county_data0 = texas_jails_mostrecent.join(texas_pop_covid_mostrecent.set_index('county'), on='county', rsuffix='_countypop_covid')

# compute infection and death rates in the county
texas_county_data0['infection_rate_countypop'] = texas_county_data0['cases'] / texas_county_data0['resident_population']
texas_county_data0['death_rate_countypop'] = texas_county_data0['deaths'] / texas_county_data0['resident_population']

# california
# merge
california_county_data0 = california_jails_mostrecent.join(california_pop_covid_mostrecent.set_index('county'), on='county', rsuffix='_countypop_covid')

# compute infection and death rates in the county
california_county_data0['infection_rate_countypop'] = california_county_data0['cases'] / california_county_data0['resident_population']
california_county_data0['death_rate_countypop'] = california_county_data0['deaths'] / california_county_data0['resident_population']


# ohio
# merge
ohio_county_data0 = ohio_jails_mostrecent.join(ohio_pop_covid_mostrecent.set_index('county'), on='county', rsuffix='_countypop_covid')

# compute infection and death rates in the county
ohio_county_data0['infection_rate_countypop'] = ohio_county_data0['cases'] / ohio_county_data0['resident_population']
ohio_county_data0['death_rate_countypop'] = ohio_county_data0['deaths'] / ohio_county_data0['resident_population']


# oregon
# merge
oregon_county_data0 = oregon_jails_mostrecent.join(oregon_pop_covid_mostrecent.set_index('county'), on='county', rsuffix='_countypop_covid')

# compute infection and death rates in the county
oregon_county_data0['infection_rate_countypop'] = oregon_county_data0['cases'] / oregon_county_data0['resident_population']
oregon_county_data0['death_rate_countypop'] = oregon_county_data0['deaths'] / oregon_county_data0['resident_population']

In [285]:
texas_county_data0.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop
65622,48001,2020-05-01,138.0,Anderson,Texas,rural,Anderson County Sheriff's Office,57735.0,239.0,2020-05-11,41.0,0.0,0.000710,0.0
65627,48003,2020-05-01,28.0,Andrews,Texas,rural,Andrews County Jail,18705.0,149.7,2020-05-11,21.0,0.0,0.001123,0.0
65652,48005,2020-05-01,160.0,Angelina,Texas,rural,Angelina County Sheriffs Department,86715.0,184.5,2020-05-11,100.0,0.0,0.001153,0.0
65657,48007,2020-05-01,140.0,Aransas,Texas,small/mid,Aransas County Jail,23510.0,595.5,2020-05-11,2.0,0.0,0.000085,0.0
65761,48009,2020-05-01,20.0,Archer,Texas,small/mid,Archer County Jail,8553.0,233.8,NaN,NaN,NaN,NaN,NaN


In [196]:
california_county_data0.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop
6358,6001,2020-04-30,1757.0,Alameda,California,urban,Alameda County Sheriffs Office,1671329.0,105.1,2020-05-11,2122,75,0.001270,0.000045
6403,6005,2020-05-01,52.0,Amador,California,rural,Amador County Sheriff's Office,39752.0,130.8,2020-05-11,7,0,0.000176,0.000000
6514,6017,2020-05-11,307.0,El Dorado,California,suburban,El Dorado County Sheriff's Office,192843.0,159.2,2020-05-11,56,0,0.000290,0.000000
6551,6019,2020-05-11,2250.0,Fresno,California,small/mid,Fresno County Sheriff Coroner's Office,999101.0,225.2,2020-05-11,945,9,0.000946,0.000009
6668,6031,2020-05-11,436.0,Kings,California,small/mid,Kings County Sheriffs Office,152940.0,285.1,2020-05-11,319,1,0.002086,0.000007


In [273]:
ohio_county_data0.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop
56931,39001,2020-05-12,20.0,Adams,Ohio,rural,Adams County Jail,27698.0,72.2,2020-05-11,6,0,0.000217,0.000000
56976,39009,2020-05-12,126.0,Athens,Ohio,rural,Southeastern Ohio Regional Jail,65327.0,192.9,2020-05-11,6,1,0.000092,0.000015
57002,39013,2020-05-01,106.0,Belmont,Ohio,small/mid,Belmont County Jail,67006.0,158.2,2020-05-11,258,7,0.003850,0.000104
57103,39025,2020-05-01,255.0,Clermont,Ohio,suburban,Clermont County Sheriffs Office,206428.0,123.5,2020-05-11,125,3,0.000606,0.000015
57217,39027,2020-05-01,50.0,Clinton,Ohio,rural,Clinton County Jail,41968.0,119.1,2020-05-11,35,0,0.000834,0.000000


In [295]:
oregon_county_data0.head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop
59885,41001,2020-05-01,14.0,Baker,Oregon,rural,Baker County Jail,16124.0,86.8,2020-05-11,1,0,0.000062,0.000000
59933,41003,2020-02-17,53.0,Benton,Oregon,small/mid,Benton County Sheriffs Office,93053.0,57.0,2020-05-11,49,5,0.000527,0.000054
60020,41005,2020-05-01,137.0,Clackamas,Oregon,suburban,Clackamas County Sheriffs Office,418187.0,32.8,2020-05-11,268,9,0.000641,0.000022
60139,41007,2020-05-01,32.0,Clatsop,Oregon,rural,Clatsop County Corrections Facility,40224.0,79.6,2020-05-11,33,0,0.000820,0.000000
60231,41009,2020-05-01,92.0,Columbia,Oregon,suburban,Columbia County Jail,52354.0,175.7,2020-05-11,15,0,0.000287,0.000000


#### Texas: (Vera Institute + NYTimes ) + TCJS  (outer join)

In [0]:
#merge
texas_county_data1 = texas_county_data0.join( texas_jail_covid.set_index('county'), on='county', rsuffix='_tcjs', how='outer')

# compute infection rates among inmates
texas_county_data1['infection_rate_inmates'] = texas_county_data1['inmates_active_positive_test_confirmation'] / texas_county_data1['jail_population']

In [234]:
texas_county_data1.sort_values(by=['infection_rate_inmates'], ascending=False).head()

,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop,date_tcjs,inmates_active_positive_test_confirmation,inmates_pending_test_results,inmates_quarantined,inmates_treated_offside,inmates_deaths_confirmed,inmates_deaths_suspected,staff_active_positive_test,staff_quarantined_pending_test_results,infection_rate_inmates
67510,48201,2020-05-12,7492.0,Harris,Texas,urban,"Baytown City Jail, Harris County Sheriff's Office",4713325.0,159.0,2020-05-11,8176.0,179.0,0.001735,0.000038,2020-05-12,669.0,5.0,2511.0,14.0,0.0,1.0,193.0,56.0,0.089295
67525,48207,2020-05-01,437.0,Haskell,Texas,rural,Rolling Plains Detention Center,5658.0,7723.6,NaN,NaN,NaN,NaN,NaN,2020-05-12,35.0,7.0,0.0,0.0,0.0,0.0,0.0,3.0,0.080092
65937,48029,2020-05-01,3341.0,Bexar,Texas,urban,Bexar County Sheriff's Office,2003554.0,166.8,2020-05-11,1920.0,57.0,0.000958,0.000028,2020-05-12,208.0,124.0,1185.0,0.0,1.0,0.0,41.0,581.0,0.062257
66749,48113,2020-05-01,4829.0,Dallas,Texas,urban,"Dallas County Sheriff's Department, Irving Cit...",2635516.0,183.2,2020-05-11,6123.0,145.0,0.002323,0.000055,2020-05-12,233.0,9.0,824.0,0.0,0.0,0.0,47.0,13.0,0.048250
70323,48479,2020-05-01,345.0,Webb,Texas,small/mid,Webb County Jail,276652.0,124.7,2020-05-11,437.0,17.0,0.001580,0.000061,2020-05-12,15.0,1.0,228.0,0.0,0.0,0.0,9.0,0.0,0.043478


In [235]:
texas_county_data1[['fips', 
                    'date', 
                    'jail_population', 
                    'county', 
                    'resident_population', 
                    'infection_rate_countypop', 
                    'infection_rate_inmates']].sort_values(by=['infection_rate_inmates'], ascending=False)

,fips,date,jail_population,county,resident_population,infection_rate_countypop,infection_rate_inmates
67510,48201,2020-05-12,7492.0,Harris,4713325.0,0.001735,0.089295
67525,48207,2020-05-01,437.0,Haskell,5658.0,NaN,0.080092
65937,48029,2020-05-01,3341.0,Bexar,2003554.0,0.000958,0.062257
66749,48113,2020-05-01,4829.0,Dallas,2635516.0,0.002323,0.048250
70323,48479,2020-05-01,345.0,Webb,276652.0,0.001580,0.043478
...,...,...,...,...,...,...,...
70601,48499,2020-05-01,120.0,Wood,45539.0,0.000307,NaN
70606,48501,2020-05-01,25.0,Yoakum,8713.0,0.000230,NaN
70719,48503,2020-05-11,20.0,Young,18010.0,0.000222,NaN
70724,48505,2020-05-01,99.0,Zapata,14179.0,0.000494,NaN


#### Texas: (Vera Institute + NYTimes + TCJS) + Releases Info  (outer join)

In [0]:
texas_county_data2 = texas_county_data1.join(texas_jail_releases.set_index('county'), on='county', how='outer').reset_index()

In [237]:
texas_county_data2

,index,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop,date_tcjs,inmates_active_positive_test_confirmation,inmates_pending_test_results,inmates_quarantined,inmates_treated_offside,inmates_deaths_confirmed,inmates_deaths_suspected,staff_active_positive_test,staff_quarantined_pending_test_results,infection_rate_inmates,State,Facility,Authorizing Agent,Known Capacity \n(if available),prior_pop,releases,frac_reduction
0,65622,48001,2020-05-01,138.0,Anderson,Texas,rural,Anderson County Sheriff's Office,57735.0,239.0,2020-05-11,41.0,0.0,0.000710,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,65627,48003,2020-05-01,28.0,Andrews,Texas,rural,Andrews County Jail,18705.0,149.7,2020-05-11,21.0,0.0,0.001123,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,65652,48005,2020-05-01,160.0,Angelina,Texas,rural,Angelina County Sheriffs Department,86715.0,184.5,2020-05-11,100.0,0.0,0.001153,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,65657,48007,2020-05-01,140.0,Aransas,Texas,small/mid,Aransas County Jail,23510.0,595.5,2020-05-11,2.0,0.0,0.000085,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65761,48009,2020-05-01,20.0,Archer,Texas,small/mid,Archer County Jail,8553.0,233.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,70601,48499,2020-05-01,120.0,Wood,Texas,rural,Wood County Sheriffs Office,45539.0,263.5,2020-05-11,14.0,0.0,0.000307,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,70606,48501,2020-05-01,25.0,Yoakum,Texas,rural,Yoakum County Law Enforcement Center,8713.0,286.9,2020-05-11,2.0,0.0,0.000230,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,70719,48503,2020-05-11,20.0,Young,Texas,rural,Young County Jail,18010.0,111.0,2020-05-11,4.0,1.0,0.000222,0.000056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,70724,48505,2020-05-01,99.0,Zapata,Texas,rural,Zapata County Jail,14179.0,698.2,2020-05-11,7.0,0.0,0.000494,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### California: (Vera + NYTimes) + County Jail Covid Cases + Jail Releases data 

In [0]:
# Vera + NYT + County Jail covid cases
#   we only have one county jail covid cases, LA county
california_county_data1 = california_county_data0.join( california_jail_covid.set_index('county'), on='county', rsuffix='_cases', how='outer').reset_index()


# compute infection rates among inmates
california_county_data1['infection_rate_inmates'] = california_county_data1['inmates_confirmed'] / california_county_data1['jail_population']

In [220]:
california_county_data1.head()

,index,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop,state_cases,date_cases,staff_confirmed,inmates_confirmed,staff_deaths,inmates_deaths,staff_recovered,inmates_recovered,infection_rate_inmates
0,6358,6001,2020-04-30,1757.0,Alameda,California,urban,Alameda County Sheriffs Office,1671329.0,105.1,2020-05-11,2122,75,0.001270,0.000045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6403,6005,2020-05-01,52.0,Amador,California,rural,Amador County Sheriff's Office,39752.0,130.8,2020-05-11,7,0,0.000176,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6514,6017,2020-05-11,307.0,El Dorado,California,suburban,El Dorado County Sheriff's Office,192843.0,159.2,2020-05-11,56,0,0.000290,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6551,6019,2020-05-11,2250.0,Fresno,California,small/mid,Fresno County Sheriff Coroner's Office,999101.0,225.2,2020-05-11,945,9,0.000946,0.000009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6668,6031,2020-05-11,436.0,Kings,California,small/mid,Kings County Sheriffs Office,152940.0,285.1,2020-05-11,319,1,0.002086,0.000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Vera + NYT + County Jail covid cases + Jail releases
# vera doesn't have jail info in San Diego and San Mateo
#    but UCLA spreadsheet has releases info on San Diego and San Mateo
california_county_data2 = california_county_data1.join( california_jail_releases.set_index('county'), on='county', rsuffix='_releases', how='outer').reset_index()

In [223]:
california_county_data2

,level_0,index,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop,state_cases,date_cases,staff_confirmed,inmates_confirmed,staff_deaths,inmates_deaths,staff_recovered,inmates_recovered,infection_rate_inmates,State,Facility,Authorizing Agent,Known Capacity \n(if available),prior_pop,releases,frac_reduction
0,0.0,6358.0,6001.0,2020-04-30,1757.0,Alameda,California,urban,Alameda County Sheriffs Office,1671329.0,105.1,2020-05-11,2122.0,75.0,0.001270,0.000045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,California,Santa Rita,Court,NaN,2500.0,247.0,0.098800
1,0.0,6358.0,6001.0,2020-04-30,1757.0,Alameda,California,urban,Alameda County Sheriffs Office,1671329.0,105.1,2020-05-11,2122.0,75.0,0.001270,0.000045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,California,NaN,NaN,NaN,2500.0,500.0,0.200000
2,1.0,6403.0,6005.0,2020-05-01,52.0,Amador,California,rural,Amador County Sheriff's Office,39752.0,130.8,2020-05-11,7.0,0.0,0.000176,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,6514.0,6017.0,2020-05-11,307.0,El Dorado,California,suburban,El Dorado County Sheriff's Office,192843.0,159.2,2020-05-11,56.0,0.0,0.000290,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,6551.0,6019.0,2020-05-11,2250.0,Fresno,California,small/mid,Fresno County Sheriff Coroner's Office,999101.0,225.2,2020-05-11,945.0,9.0,0.000946,0.000009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,6668.0,6031.0,2020-05-11,436.0,Kings,California,small/mid,Kings County Sheriffs Office,152940.0,285.1,2020-05-11,319.0,1.0,0.002086,0.000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,6685.0,6033.0,2020-05-01,174.0,Lake,California,rural,Lake County Sheriff's Office,64386.0,270.2,2020-05-11,8.0,0.0,0.000124,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6.0,6719.0,6037.0,2020-05-11,11723.0,Los Angeles,California,urban,Los Angeles County Sheriff's Department,10039107.0,116.8,2020-05-11,32258.0,1569.0,0.003213,0.000156,California,5/11/20,107,357.0,0.0,0.0,NaN,117.0,0.030453,California,County-wide,NaN,NaN,17076.0,5193.0,0.304111
8,7.0,6765.0,6039.0,2020-05-01,530.0,Madera,California,small/mid,Madera County Department Of Corrections,157327.0,336.9,2020-05-11,67.0,2.0,0.000426,0.000013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,8.0,6804.0,6041.0,2020-05-12,145.0,Marin,California,suburban,Marin County Sheriffs Office,258826.0,56.0,2020-05-11,267.0,14.0,0.001032,0.000054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,California,County jail,Court/others,NaN,270.0,58.0,0.214815


#### (Vera Institute + NYTimes) + Releases Info  (inner join)

In [0]:
texas_county_data2b = texas_county_data0.join(texas_jail_releases.set_index('county'), on='county', how='right').reset_index()

In [84]:
texas_county_data2b

,index,fips,date,jail_population,county,state,urbanicity,reporting_jurisdictions,resident_population,jail_incarceration_rate_per_100k,date_countypop_covid,cases,deaths,infection_rate_countypop,death_rate_countypop,State,Facility,Authorizing Agent,Known Capacity \n(if available),prior_pop,releases,frac_reduction
0,66749,48113,2020-05-01,4829.0,Dallas,Texas,urban,"Dallas County Sheriff's Department, Irving Cit...",2635516.0,183.2,2020-05-11,6123.0,145.0,0.002323,0.000055,Texas,County-wide,Courts,NaN,5879,1000,0.170097
1,67510,48201,2020-05-12,7492.0,Harris,Texas,urban,"Baytown City Jail, Harris County Sheriff's Office",4713325.0,159.0,2020-05-11,8176.0,179.0,0.001735,0.000038,Texas,County-wide,NaN,NaN,9059,1789,0.197483
2,70074,48453,2020-05-01,1698.0,Travis,Texas,urban,Travis County Sheriff's Office,1273954.0,133.3,2020-05-11,2171.0,65.0,0.001704,0.000051,Texas,County-wide,courts,NaN,2164,97,0.044824


## Other references


+ US-wide data
  + https://www.bjs.gov/content/pub/pdf/ji16.pdf
+ State-level
  + Texas 
    + county-level jail data: https://www.tcjs.state.tx.us/wp-content/uploads/2020/05/TCJS_COVID_Report.pdf
    + state-level jail data: https://www.prisonpolicy.org/profiles/TX.html
    + "Today, the Texas Department of Criminal Justice (TDCJ) oversees 17 state jails, 14 directly and three through private contractors, in 16 counties throughout the state (Exhibit 2)."  (https://comptroller.texas.gov/economy/fiscal-notes/2019/aug/jails.php)
    + http://www.prisonpro.com/content/texas-prisons-jails
    + https://www.tdcj.texas.gov/covid-19/index.html
      + Texas Department of Criminal Justice
      + Starting this week (May 12): "Testing is one key to stopping the spread of COVID-19. Thanks to Texas Governor Greg Abbott and with the help of the Texas Department of Emergency Management, the Texas Department of Criminal Justice (TDCJ) this week is beginning a new expanded testing program."
      + I think this is mainly state prison data: In the first few hours of testing nearly 1500 tests were administered and collected. Like has occurred in virtually every area where large scale asymptomatic testing has taken place TDCJ fully expects the number of positive offender cases to expand dramatically. The agency has a plan in place to medically restrict these new asymptomatic cases as needed.  In total there are 652 TDCJ employees, staff or contractors who have tested positive for COVID-19 and 1733 offenders who have tested positive. There have been 30 confirmed offender deaths and 23 pending autopsy results. 7 staff members have died from COVID-19. There are now 91 employees and 371 offenders who have medically recovered from COVID-19.  19,214 offenders are on medical restriction because they may have had contact with either an employee or offender with a positive or pending COVID-19 test.
  + Michigan:
    + Release info:
      + Ogemaw County: https://www.ogemawherald.com/stories/jail-releases-certain-inmates-due-to-covid-19-pandemic,109566
      + Governor's Executive order (April 26): https://www.michigan.gov/whitmer/0,9309,7-387-90499_90705-523422--,00.html
      + Metro Detroit County Jails: https://www.clickondetroit.com/news/local/2020/04/02/hundreds-of-inmates-released-from-metro-detroit-county-jails-amid-coronavirus-covid-19-outbreak/
      + Washtenaw county jail: https://www.michiganradio.org/post/aggressive-covid-19-plan-washtenaw-county-jail-reduces-population-more-half
      + Calhoun County: https://myemail.constantcontact.com/Press-release--First-Inmate-Identified-With-COVID-19.html?soid=1131086442739&aid=DXtI3PWzW68


# Texas - Summary

+ From merged datasets
+ Not a time series -- uses most recent dates in each dataset

In [241]:
# displaying top 15 counties by infection rate among inmates, out of >200 counties
#   only 10 counties has inmate infection rate data
#   only 3 counties has jail releases data
texas_county_data2[['fips', 
                    'county', 
                    'date', 
                    'jail_population', 
                    'jail_incarceration_rate_per_100k',
                    'resident_population', 
                    'infection_rate_countypop', 
                    'infection_rate_inmates',
                    'prior_pop',
                    'releases',
                    'frac_reduction',
                    'date_countypop_covid',
                    'date_tcjs']].sort_values(by=['infection_rate_inmates'], ascending=False).head(15)

,fips,county,date,jail_population,jail_incarceration_rate_per_100k,resident_population,infection_rate_countypop,infection_rate_inmates,prior_pop,releases,frac_reduction,date_countypop_covid,date_tcjs
100,48201,Harris,2020-05-12,7492.0,159.0,4713325.0,0.001735,0.089295,9059.0,1789.0,0.197483,2020-05-11,2020-05-12
103,48207,Haskell,2020-05-01,437.0,7723.6,5658.0,NaN,0.080092,NaN,NaN,NaN,NaN,2020-05-12
14,48029,Bexar,2020-05-01,3341.0,166.8,2003554.0,0.000958,0.062257,NaN,NaN,NaN,2020-05-11,2020-05-12
56,48113,Dallas,2020-05-01,4829.0,183.2,2635516.0,0.002323,0.048250,5879.0,1000.0,0.170097,2020-05-11,2020-05-12
239,48479,Webb,2020-05-01,345.0,124.7,276652.0,0.001580,0.043478,NaN,NaN,NaN,2020-05-11,2020-05-12
219,48439,Tarrant,2020-05-01,3494.0,166.2,2102515.0,0.001781,0.016600,NaN,NaN,NaN,2020-05-11,2020-05-12
211,48423,Smith,2020-05-01,755.0,324.4,232751.0,0.000765,0.013245,NaN,NaN,NaN,2020-05-11,2020-05-12
91,48183,Gregg,2020-05-01,587.0,473.6,123945.0,0.000952,0.006814,NaN,NaN,NaN,2020-05-11,2020-05-12
169,48339,Montgomery,2020-05-01,612.0,100.8,607391.0,0.001192,0.006536,NaN,NaN,NaN,2020-05-11,2020-05-12
73,48147,Fannin,2020-05-01,506.0,1424.8,35514.0,0.000648,0.003953,NaN,NaN,NaN,2020-05-11,2020-05-12


# California - Summary


+ From merged datasets
+ Not a time series -- uses most recent dates in each dataset

In [225]:
california_county_data2[['fips', 
                    'county', 
                    'date', 
                    'jail_population', 
                    'jail_incarceration_rate_per_100k',
                    'resident_population', 
                    'infection_rate_countypop', 
                    'infection_rate_inmates',
                    'prior_pop',
                    'releases',
                    'frac_reduction',
                    'date_countypop_covid',
                    'date_cases']]

,fips,county,date,jail_population,jail_incarceration_rate_per_100k,resident_population,infection_rate_countypop,infection_rate_inmates,prior_pop,releases,frac_reduction,date_countypop_covid,date_cases
0,6001.0,Alameda,2020-04-30,1757.0,105.1,1671329.0,0.001270,NaN,2500.0,247.0,0.098800,2020-05-11,NaN
1,6001.0,Alameda,2020-04-30,1757.0,105.1,1671329.0,0.001270,NaN,2500.0,500.0,0.200000,2020-05-11,NaN
2,6005.0,Amador,2020-05-01,52.0,130.8,39752.0,0.000176,NaN,NaN,NaN,NaN,2020-05-11,NaN
3,6017.0,El Dorado,2020-05-11,307.0,159.2,192843.0,0.000290,NaN,NaN,NaN,NaN,2020-05-11,NaN
4,6019.0,Fresno,2020-05-11,2250.0,225.2,999101.0,0.000946,NaN,NaN,NaN,NaN,2020-05-11,NaN
5,6031.0,Kings,2020-05-11,436.0,285.1,152940.0,0.002086,NaN,NaN,NaN,NaN,2020-05-11,NaN
6,6033.0,Lake,2020-05-01,174.0,270.2,64386.0,0.000124,NaN,NaN,NaN,NaN,2020-05-11,NaN
7,6037.0,Los Angeles,2020-05-11,11723.0,116.8,10039107.0,0.003213,0.030453,17076.0,5193.0,0.304111,2020-05-11,5/11/20
8,6039.0,Madera,2020-05-01,530.0,336.9,157327.0,0.000426,NaN,NaN,NaN,NaN,2020-05-11,NaN
9,6041.0,Marin,2020-05-12,145.0,56.0,258826.0,0.001032,NaN,270.0,58.0,0.214815,2020-05-11,NaN


# Ohio - Summary

In [297]:
ohio_county_data0[['fips', 
                    'county', 
                    'date', 
                    'jail_population', 
                    'jail_incarceration_rate_per_100k',
                    'resident_population', 
                    'infection_rate_countypop',
                    'date_countypop_covid']]

,fips,county,date,jail_population,jail_incarceration_rate_per_100k,resident_population,infection_rate_countypop,date_countypop_covid
56931,39001,Adams,2020-05-12,20.0,72.2,27698.0,0.000217,2020-05-11
56976,39009,Athens,2020-05-12,126.0,192.9,65327.0,0.000092,2020-05-11
57002,39013,Belmont,2020-05-01,106.0,158.2,67006.0,0.003850,2020-05-11
57103,39025,Clermont,2020-05-01,255.0,123.5,206428.0,0.000606,2020-05-11
57217,39027,Clinton,2020-05-01,50.0,119.1,41968.0,0.000834,2020-05-11
57265,39031,Coshocton,2020-05-01,44.0,120.2,36600.0,0.000519,2020-05-11
57384,39041,Delaware,2020-05-01,118.0,56.4,209177.0,0.001061,2020-05-11
57503,39043,Erie,2020-05-01,59.0,79.4,74266.0,0.001118,2020-05-11
57622,39049,Franklin,2020-05-01,1346.0,102.2,1316756.0,0.002947,2020-05-11
57728,39053,Gallia,2020-05-01,32.0,107.0,29898.0,0.000201,2020-05-11


# Oregon - Summary

In [299]:
oregon_county_data0[['fips', 
                    'county', 
                    'date', 
                    'jail_population', 
                    'jail_incarceration_rate_per_100k',
                    'resident_population', 
                    'infection_rate_countypop',
                    'date_countypop_covid']]

,fips,county,date,jail_population,jail_incarceration_rate_per_100k,resident_population,infection_rate_countypop,date_countypop_covid
59885,41001,Baker,2020-05-01,14.0,86.8,16124.0,0.000062,2020-05-11
59933,41003,Benton,2020-02-17,53.0,57.0,93053.0,0.000527,2020-05-11
60020,41005,Clackamas,2020-05-01,137.0,32.8,418187.0,0.000641,2020-05-11
60139,41007,Clatsop,2020-05-01,32.0,79.6,40224.0,0.000820,2020-05-11
60231,41009,Columbia,2020-05-01,92.0,175.7,52354.0,0.000287,2020-05-11
60239,41011,Coos,2020-05-01,32.0,49.6,64487.0,0.000465,2020-05-11
60353,41013,Crook,2020-05-12,20.0,82.0,24404.0,0.000041,2020-05-11
60390,41015,Curry,2020-05-01,25.0,109.1,22925.0,0.000174,2020-05-11
60508,41017,Deschutes,2020-05-12,142.0,71.8,197692.0,0.000435,2020-05-11
60634,41019,Douglas,2020-05-12,72.0,64.9,110980.0,0.000216,2020-05-11
